In [1]:
import geopandas as gpd
import requests
import zipfile
import io
import matplotlib.pyplot as plt
import os
import fiona
import pandas as pd
import sys
#%matplotlib inline  # jupyter "magic" to display plots in notebook
pd.set_option('mode.chained_assignment', None)
import numpy as np
from bs4 import BeautifulSoup
import re
pd.options.display.max_columns = None
pd.options.mode.chained_assignment = None  # default='warn'
from geojson import Point, Feature, FeatureCollection, dump

import sys
sys.path.append("../")
import src.soporteScrapping as scr

parameters

In [2]:
url = 'https://download.geofabrik.de/europe/italy/'
country = 'IT'
country_clean = 'IT'

get links

In [3]:
links,flattened = scr.getLinks(url)
links

200


['https://download.geofabrik.de/europe/italy/centro-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/isole-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/nord-ovest-latest-free.shp.zip',
 'https://download.geofabrik.de/europe/italy/sud-latest-free.shp.zip']

In [4]:
flattened

['centro-latest-free.shp.zip',
 'isole-latest-free.shp.zip',
 'nord-est-latest-free.shp.zip',
 'nord-ovest-latest-free.shp.zip',
 'sud-latest-free.shp.zip']

data conversion table input

In [5]:
ruta = os.path.dirname(os.getcwd()) + '\\data'

In [6]:
inPOIS = pd.ExcelFile(f'{ruta}\\POIS_Classification' + '.xlsx')
sheets_list = inPOIS.sheet_names[0]
POIS = pd.read_excel(inPOIS, sheet_name = sheets_list)
POIS.rename({'Tipo2':'Subtipo', 'Subtipo':'fclass'}, axis = 1, inplace=True)
POIS.head()

c:\Users\annas\anaconda3\envs\iron\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,Tipo,Subtipo,fclass
0,Accommodation,Hospitality,alpine_hut
1,Accommodation,Hospitality,camp_site
2,Accommodation,Hospitality,chalet
3,Accommodation,Hospitality,guesthouse
4,Accommodation,Hospitality,hostel


downloading shapefile

In [7]:
print('Downloading shapefile...')
patron = r'(.*).shp.zip'
for link, flat in zip(links, flattened):
    local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_{country}'
    r = requests.get(link)
    z = zipfile.ZipFile(io.BytesIO(r.content), mode='a')
    folder = re.findall(patron, flat)
    os.mkdir(os.path.join(local_path, folder[0]))
    z.extractall(path=(local_path + '//'+ folder[0])) # extract to folder
    print(f"Done with {link}")
filenames = [y for y in sorted(z.namelist()) for starting in ['gis_osm_landuse_a_free_1', 'gis_osm_buildings_a_free_1'] if y.startswith(starting)] 
print(filenames)

Done with https://download.geofabrik.de/europe/italy/centro-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/isole-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/nord-est-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/nord-ovest-latest-free.shp.zip
Done with https://download.geofabrik.de/europe/italy/sud-latest-free.shp.zip
['gis_osm_buildings_a_free_1.cpg', 'gis_osm_buildings_a_free_1.dbf', 'gis_osm_buildings_a_free_1.prj', 'gis_osm_buildings_a_free_1.shp', 'gis_osm_buildings_a_free_1.shx', 'gis_osm_landuse_a_free_1.cpg', 'gis_osm_landuse_a_free_1.dbf', 'gis_osm_landuse_a_free_1.prj', 'gis_osm_landuse_a_free_1.shp', 'gis_osm_landuse_a_free_1.shx']


POIs

In [7]:
local_path = os.path.dirname(os.getcwd()) + f'\\data\\tmpOSM_{country}'

In [8]:
alldf_pois = []
for subdir, dirs, files in os.walk(local_path):
    #global alldf_landuse
    for file in files:
        #print(file)
        if file.startswith('gis_osm_pois_free_1.shp'):
            print(file, subdir)
            df_pois = gpd.read_file(subdir + '\\' + 'gis_osm_pois_free_1' + '.shp')
            alldf_pois.append(df_pois)

gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\centro-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\isole-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\nord-est-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\nord-ovest-latest-free
gis_osm_pois_free_1.shp e:\Cursos\00ironhack_DA\00LABS\vuelve-conmigo-a-italia\data\tmpOSM_IT\sud-latest-free


In [9]:
df_pois = pd.concat([(alldf_pois[n]) for n in list(range(len(alldf_pois)))])

In [10]:
print(df_pois.shape)
df_pois.head()

(727951, 5)


,osm_id,code,fclass,name,geometry
0,21027591,2733,archaeological,Via cava del Cavone,POINT (11.63634 42.66276)
1,26859360,2422,camp_site,Albinia,POINT (10.52806 43.01722)
2,26859373,2422,camp_site,Argentario Camping Village,POINT (11.19245 42.49649)
3,26859377,2422,camp_site,Camping Village Assisi,POINT (12.57420 43.07560)
4,26859385,2422,camp_site,Badiaccia,POINT (12.01639 43.18028)


reproject

In [11]:
df_pois.crs ## tiene que estar en WGS 84

<Geographic 2D CRS: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84" ...>
Name: WGS 84
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [12]:
df_pois.to_crs("EPSG:4326", inplace=True)

In [13]:
df_pois.crs ## tiene que estar en WGS 84

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

spatial join geoLevels

In [14]:
GeoLevels_IT_03 = gpd.read_file(ruta + '\\' + 'GeoLevels_IT_03.geojson')
GeoLevels_IT_03.to_crs("EPSG:4326", inplace = True)

In [15]:
df_pois_merge = df_pois.sjoin(GeoLevels_IT_03, how="inner")
print(df_pois_merge.shape)
df_pois_merge.head(1)

(33079, 16)


,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio
6,26860774,2422,camp_site,Camping Biscione,POINT (12.47814 37.70137),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino


merge with equivalence table

In [16]:
POIS.head()

,Tipo,Subtipo,fclass
0,Accommodation,Hospitality,alpine_hut
1,Accommodation,Hospitality,camp_site
2,Accommodation,Hospitality,chalet
3,Accommodation,Hospitality,guesthouse
4,Accommodation,Hospitality,hostel


In [17]:
df_pois_merge = df_pois_merge.merge(POIS, on ='fclass', how = 'left', indicator = True)
df_pois_merge['_merge'].value_counts()

both          33079
left_only         0
right_only        0
Name: _merge, dtype: int64

In [18]:
df_pois_merge.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 33079 entries, 0 to 33078
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   osm_id         33079 non-null  object  
 1   code           33079 non-null  int64   
 2   fclass         33079 non-null  object  
 3   name           21539 non-null  object  
 4   geometry       33079 non-null  geometry
 5   index_right    33079 non-null  int64   
 6   OBJECTID       33079 non-null  int64   
 7   nivel          33079 non-null  int64   
 8   cod_Pais       33079 non-null  object  
 9   nom_Pais       33079 non-null  object  
 10  cod_Region     33079 non-null  object  
 11  nom_Region     33079 non-null  object  
 12  cod_Provincia  33079 non-null  object  
 13  nom_Provincia  33079 non-null  object  
 14  cod_Municipio  33079 non-null  object  
 15  nom_Municipio  33079 non-null  object  
 16  Tipo           23614 non-null  object  
 17  Subtipo        23614 no

In [19]:
df_pois_merge.head()

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,_merge
0,26860774,2422,camp_site,Camping Biscione,POINT (12.47814 37.70137),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality,both
1,639702152,2903,drinking_water,NaN,POINT (12.47201 37.70619),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Amenity,Amenity,both
2,639702161,2903,drinking_water,NaN,POINT (12.48432 37.69290),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Amenity,Amenity,both
3,639702164,2903,drinking_water,NaN,POINT (12.49512 37.71213),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Amenity,Amenity,both
4,851680452,2501,supermarket,NaN,POINT (12.49964 37.71835),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Food shops,Grocery store,both


selecting only the data that falls into sicily

In [20]:
df_pois_merge['nom_Region'].unique()

array(['Sicilia'], dtype=object)

In [21]:
df_pois_merge = df_pois_merge[df_pois_merge['nom_Region'] == 'Sicilia']

In [22]:
df_pois_merge.drop(['_merge'], axis = 1, inplace = True)

add hotel listings to paint both of them together

In [78]:
local_path = os.path.dirname(os.getcwd()) + f'\\data'
local_path

'e:\\Cursos\\00ironhack_DA\\00LABS\\vuelve-conmigo-a-italia\\data'

In [79]:
df_booking = pd.read_csv(local_path + '//booking.csv', sep =';', decimal=',', thousands='.')
df_booking.head(1)

,name,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image,parking,air_conditioning,bathroom,wifi,breakfast,coffee_tea_maker,laundry,location,point,latitude,longitude,altitude,url_clean,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio
0,La Maison del Sole,"84 via napoli, 90133 palermo, italy",84.0,NaN,9.4,125.0,01/09/2023,https://cf.bstatic.com/xdata/images/hotel/squa...,N,N,N,N,N,N,N,"Via Napoli 84, 90133, Palermo","(38.117480016368205, 13.360619970299581, 0.0)",38.11748,13.36062,0,https://www.booking.com/hotel/it/la-maison-del...,POINT (13.36061997029958 38.117480016368205),125,1240332,3,IT,Italia,19,Sicilia,82,Palermo,82053,Palermo


turn into geodataframe and remove unnecesary columns

In [80]:
df_booking = gpd.GeoDataFrame(df_booking, geometry=gpd.points_from_xy(df_booking.longitude, df_booking.latitude))
df_booking.set_crs("EPSG:4326", inplace = True)
df_booking.head(1)

,name,address,currentPrice,previousPrice,rating,reviewNumber,checkingDate,image,parking,air_conditioning,bathroom,wifi,breakfast,coffee_tea_maker,laundry,location,point,latitude,longitude,altitude,url_clean,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio
0,La Maison del Sole,"84 via napoli, 90133 palermo, italy",84.0,NaN,9.4,125.0,01/09/2023,https://cf.bstatic.com/xdata/images/hotel/squa...,N,N,N,N,N,N,N,"Via Napoli 84, 90133, Palermo","(38.117480016368205, 13.360619970299581, 0.0)",38.11748,13.36062,0,https://www.booking.com/hotel/it/la-maison-del...,POINT (13.36062 38.11748),125,1240332,3,IT,Italia,19,Sicilia,82,Palermo,82053,Palermo


In [81]:
df_pois_merge['currenPrice'] = 100
df_pois_merge.head(1)

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,currenPrice
0,26860774,2422,camp_site,Camping Biscione,POINT (12.47814 37.70137),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality,100


In [82]:
df_booking['fclass'] = 'hotel'
df_booking['Tipo'] = 'Hotel'
df_booking['Subtipo'] = 'Hotel'
df_booking.drop(['address', 'previousPrice', 'rating', 'reviewNumber', 'image', 'parking', 'air_conditioning', 'bathroom', 'wifi', 'breakfast', 'coffee_tea_maker', 'laundry', 'location', 'point', 
                 'latitude', 'longitude', 'altitude', 'url_clean'], axis = 1, inplace = True)
df_booking.head(1)

,name,currentPrice,checkingDate,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,fclass,Tipo,Subtipo
0,La Maison del Sole,84.0,01/09/2023,POINT (13.36062 38.11748),125,1240332,3,IT,Italia,19,Sicilia,82,Palermo,82053,Palermo,hotel,Hotel,Hotel


In [83]:
df_total = pd.concat([df_pois_merge, df_booking])
print(df_total.shape)
df_total.head(1)

(45899, 21)


,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,currenPrice,currentPrice,checkingDate
0,26860774,2422.0,camp_site,Camping Biscione,POINT (12.47814 37.70137),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality,100.0,NaN,NaN


export

In [84]:
df_booking.dtypes

name               object
currentPrice      float64
checkingDate       object
geometry         geometry
index_right         int64
OBJECTID            int64
nivel               int64
cod_Pais           object
nom_Pais           object
cod_Region          int64
nom_Region         object
cod_Provincia       int64
nom_Provincia      object
cod_Municipio       int64
nom_Municipio      object
fclass             object
Tipo               object
Subtipo            object
dtype: object

In [85]:
local_path = os.path.dirname(os.getcwd()) + f'\\data'
local_path

'e:\\Cursos\\00ironhack_DA\\00LABS\\vuelve-conmigo-a-italia\\data'

In [86]:
df_total['lon'] = df_total.geometry.apply(lambda p: p.x)
df_total['lat'] = df_total.geometry.apply(lambda p: p.y)
df_total.head(1)

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,currenPrice,currentPrice,checkingDate,lon,lat
0,26860774,2422.0,camp_site,Camping Biscione,POINT (12.47814 37.70137),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality,100.0,NaN,NaN,12.478139,37.701373


In [87]:
df_total.to_file(f'{local_path}\\df_pois.geojson', driver='GeoJSON')  

In [ ]:
df_total.head(1)

,osm_id,code,fclass,name,geometry,index_right,OBJECTID,nivel,cod_Pais,nom_Pais,cod_Region,nom_Region,cod_Provincia,nom_Provincia,cod_Municipio,nom_Municipio,Tipo,Subtipo,currentPrice,lon,lat
0,26860774,2422.0,camp_site,Camping Biscione,POINT (12.47814 37.70137),321,1241383,3,IT,Italia,19,Sicilia,81,Trapani,81024,Petrosino,Accommodation,Hospitality,NaN,12.478139,37.701373


In [ ]:
df_total['currentPrice'].value_counts()

70.0     262
90.0     244
85.0     233
80.0     225
60.0     201
        ... 
342.0      1
324.0      1
327.0      1
520.0      1
331.0      1
Name: currentPrice, Length: 358, dtype: int64

In [ ]:
df_total.to_csv(f'{local_path}//pois.csv', encoding='utf-8', sep='|', decimal='.', index=False)

In [ ]:
print(df_total[df_pois_merge['nom_Region'] == 'Sicilia'].shape)

(45899, 21)


c:\Users\annas\anaconda3\envs\iron\lib\site-packages\geopandas\geodataframe.py:1415: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
